In [ ]:
from collections import defaultdict
import os
import gzip
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import Bio.UniProt.GOA as gafiterator

from utils import colname_to_mi, get_orthologs, defaultdict_to_regular, get_go_terms, plot_upset

# ignore warnings pandas for groubpy
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
# Rab name conversion to match the Rab names in Jean dataset. 
# Matched name according to this list: https://www.genenames.org/data/genegroup/#!/group/388
rab_conversion = {
                    'RAB1A': 'RAB1',
                    'RAB2A': 'RAB2',
                    'RAB7A': 'RAB7',
                    'RAB9A': 'RAB9',
                    'RAB11A': 'RAB11',
                    'RAB27A': 'RAB27',
                    'RAB4A': 'RAB4',
                    'RAB5A': 'RAB5',
                    'RAB5B': 'RAB5',
                    'RAB5C': 'RAB5',
                    'RAB6A': 'RAB6'
}

## Loading and cleaning datasets

In [ ]:
## Files path

# Datasets
JEAN = 'data/SAINT Analysis 2-3 repeats-Organized.xlsx'
GILLINGHAM2019 = 'data/elife-45916-supp1-v2.xlsx'
GILLINGHAM2014 = 'data/mmc2.xlsx'
LI = 'data/mmc3.xlsx'
WILSON = 'data/wilson.tsv'

# Orthologs lists
GILLINGHAM2014_INT_ORTHOLOGS = 'data/orthologs/gillingham2014_interactors_orthologs.xlsx'
GILLINGHAM2014_BAIT_ORTHOLOGS = 'data/orthologs/gillingham2014_baits_orthologs.xlsx'
LI_INT_ORTHOLOGS = 'data/orthologs/li2016_interactors_orthologs.xlsx'
LI_BAIT_ORTHOLOGS = 'data/orthologs/li2016_baits_orthologs.xlsx'

# Figures output
INTERACTORS_COMPARISON = lambda type_: f'figures/genes_comparison_{type_}.svg'  # type_ = 'strict' or 'total' (intersection type)
GO_TERMS_COMPARISON = lambda cat: f'figures/GOterms_{cat}.svg' # cat = 'cellular_component', 'biological_process' or 'molecular_function'
RANDOM_INTERACTORS = lambda type_: f'figures/random_genes_comparison_{type_}.svg'
GO_TERMS_RANDOM_COMPARISON = lambda cat: f'figures/random_GOterms_{cat}.svg'
GO_ANNOTATED_GENES = 'figures/GO-annotated_genes.svg'

# GO terms
OBO = 'data/GOA/go-basic.obo'
GAF = lambda species: f'data/GOA/goa_{species}.gaf.gz' # species = 'human', 'mouse', 'fly'

In [ ]:
## Jean et al.

saint_threshold = 0.95

jean_raw = pd.read_excel(JEAN, sheet_name='SAINT Analysis 2-3 repeats', usecols=['Bait', 'Prey', 'AvgSpec', 'SaintScore'])
jean_raw['Bait'] = jean_raw['Bait'].str.replace('R', 'RAB') # Change name to match other datasets
jean = jean_raw[jean_raw['SaintScore'] > saint_threshold] # Filter out low confidence interactions
jean = jean.pivot_table(index='Prey', columns='Bait', values='AvgSpec')
jean.index = jean.index.str.upper()
jean.fillna(0, inplace=True)

In [ ]:
## Gillingham et al. (2019)

wd_threshold = 10

gill2019 = pd.read_excel(GILLINGHAM2019, skiprows=range(1,8), header=1, sheet_name='Spectral counts')
gill2019.index = gill2019['Gene Name'].str.upper()

# Filter out non-rab columns and convert colnames to multiindex
gill2019 = gill2019[[col for col in gill2019.columns if re.match(r'Rab\d+', col)]]
new_colnames = [colname_to_mi(col) for col in gill2019.columns]
gill2019.columns = pd.MultiIndex.from_tuples(new_colnames)
gill2019 = gill2019.groupby(level=[0,1], axis=1).sum()

# Get WD scores to filter out low confidence interactions
gill2019_wd_scores = pd.read_excel(GILLINGHAM2019, skiprows=range(1,8), header=1, sheet_name='Mean WD Score')
gill2019_wd_scores.index = gill2019_wd_scores['Gene Name'].str.upper()
gill2019_wd_scores = gill2019_wd_scores[[col for col in gill2019_wd_scores.columns if re.match(r'Rab\d+', col)]]
new_colnames = [tuple(colname.split('  ')) for colname in gill2019_wd_scores.columns]
gill2019_wd_scores.columns = pd.MultiIndex.from_tuples(new_colnames)

# If WD < threshold, set spectral counts to 0
gill2019 = gill2019.where(gill2019_wd_scores > wd_threshold, other=0)
gill2019 = gill2019[gill2019.sum(axis=1) > 0] # Remove rows with no spectral counts

# Sum GTP-locked and GDP-locked forms
gill2019 = gill2019.groupby(level=0, axis=1).sum()
gill2019.columns = gill2019.columns.str.upper()

In [ ]:
## Gillingham et al. (2014)

threshold = 5

gill2014 = pd.read_excel(GILLINGHAM2014, skiprows=range(1, 8), header=1,
                         sheet_name='S1A - Total Spectral Counts')
# gill2014.index = gill2014['FBgn']
gill2014.index = gill2014['Symbol']

# Filter baits other than Rabs
usecols = [col for col in gill2014.columns if re.match(r'Rab\d+', col)]
gill2014 = gill2014[usecols]

# Load score to filter out low confidence interactors
gill2014_scores = pd.read_excel(GILLINGHAM2014, skiprows=range(1, 8), header=1,
                                sheet_name='S1B - S scores')
gill2014_scores.index = gill2014_scores['Symbol']
usecols = [col for col in gill2014_scores.columns if re.match(r'Rab\d+', col)]
gill2014_scores = gill2014_scores[usecols]
gill2014_scores.columns = gill2014_scores.columns.str.upper()


# Chang bait names for their human orthologs names and format it to match Jean's dataset
gill_baits_orthologs = get_orthologs(GILLINGHAM2014_BAIT_ORTHOLOGS)
gill2014.columns = gill2014.columns.map(gill_baits_orthologs)
gill2014.columns = [
    rab_conversion[rab] if rab in rab_conversion else rab for rab in gill2014.columns
]

# Filter out low confidence interactors
gill2014 = gill2014.where(gill2014_scores > threshold, other=0)
gill2014 = gill2014[gill2014.sum(axis=1) > 0] # Remove rows with no spectral counts


In [ ]:
## Li et al. (2016)
li = pd.read_excel(LI, skiprows=1, header=0, sheet_name='Bait-prey information')
li = li.pivot_table(index=['Official Symbol'], columns=['Bait'], values='Average Spectal Counts')

# Change mouse rab names for their human orthologs names and format it to match Jean's dataset
li_bait_orthologs = get_orthologs(LI_BAIT_ORTHOLOGS)
li.columns = li.columns.map(li_bait_orthologs)
li.columns = [
    rab_conversion[rab] if rab in rab_conversion else rab for rab in li.columns
    ]
li.fillna(0, inplace=True)

## Comparing our study with Gillingham et al. (2014 and 2019) and Li et al.

In [ ]:
# Get Rab which are common to all datasets
common = list(set.intersection(
    *map(set, [jean.columns, gill2019.columns, gill2014.columns, li.columns])
))

# Clean datasets from RAB not common to all datasets
filter_df = lambda df, common: df[df[common].sum(axis=1) != 0]

jean = filter_df(jean, common)
gill2019 = filter_df(gill2019, common)
gill2014 = filter_df(gill2014, common)
li = filter_df(li, common)

common

### Interactors across studies

In [ ]:
# Convert Li and Gillingham 2014 gene names to their human orthologs

## Gillingham et al. (2014)
gill_int_orthologs = get_orthologs(GILLINGHAM2014_INT_ORTHOLOGS)
gill2014_orthologs = gill2014[gill2014.index.isin(gill_int_orthologs)]  # Remove genes with no human orthologs
gill2014_orthologs.index = gill2014_orthologs.index.map(gill_int_orthologs).str.upper()   # Change gene names to human orthologs

## Li et al. (2016)
li_int_orthologs = get_orthologs(LI_INT_ORTHOLOGS)
li_orthologs = li[li.index.isin(li_int_orthologs)]  # Remove genes with no human orthologs
li_orthologs.index = li_orthologs.index.map(li_int_orthologs).str.upper()  # Change gene names to human orthologs

print(f'{len(gill2014) - len(gill2014_orthologs)} rows on {len(gill2014)} removed from Gillingham 2014 dataset due to lack of human orthologs')
print(f'{len(li) - len(li_orthologs)} row on {len(li)} removed from Li dataset due to lack of human orthologs')

In [ ]:
# Create dict of datasets for simpler data handling
genes_datasets = {'This study': jean,
                  'Gillingham et al. (2019)': gill2019,
                  'Gillingham et al. (2014)': gill2014_orthologs,
                  'Li et al. (2016)': li_orthologs}

# Plot upset of common interactors
genes_data = {dataset: list(np.unique(df.index.tolist())) for dataset, df in genes_datasets.items()}

fig = plot_upset(genes_data, intersection_type='total')
# plt.savefig(INTERACTORS_COMPARISON('total'), bbox_inches='tight')

### GO analysis

In [ ]:
# If GO terms and GO annotations are not downloaded, download them
if len(os.listdir('data/GOA')) == 0:
    from download_GO import download
    download()

In [ ]:
# Get all interactors to extract GO terms from annotations
interactors = {'human': list(jean.index.tolist() + \
                             gill2019.index.tolist() + \
                             gill2014_orthologs.index.tolist()),
               'mouse': li.index.tolist(),
               'fly': gill2014.index.tolist()}

In [ ]:
# Dict to store Go id for each interactors for each GO term category
goa = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

# Get GO annotations for the identified interactors
for species, interacts in interactors.items():
    with gzip.open(GAF(species), 'rt') as f:
        for annotation in gafiterator.gafiterator(f):
            if annotation['DB_Object_Symbol'] not in interacts:
                continue
            
            aspect = annotation['Aspect']
            symbol = annotation['DB_Object_Symbol']
            if annotation['GO_ID'] not in goa[species][aspect][symbol]:
                goa[species][aspect][symbol].append(annotation['GO_ID'])
                
goa = defaultdict_to_regular(goa)

In [ ]:
# Extract GO terms for each dataset and each GO terms category and make an upset plot
go_categories = {
        # 'F': 'molecular_function',
        # 'C': 'cellular_component',
        'P': 'biological_process'
        }

for category in go_categories:
    go_data = {
            'This study': get_go_terms(goa['human'][category], jean.index),
            'Gillingham et al. (2019)': get_go_terms(goa['human'][category], gill2019.index),
            'Gillingham et al. (2014) *orthologs': get_go_terms(goa['human'][category], gill2014_orthologs.index),
            'Li et al. (2016)': get_go_terms(goa['mouse'][category], li.index),
            }
    
    fig = plot_upset(go_data, title=go_categories[category])
#     plt.savefig(GO_TERMS_COMPARISON(go_categories[category]), bbox_inches='tight')

# Comparing our study with Wilson et al., 2023

In [ ]:
## Jean et al.

saint_threshold = 0.95

jean_raw = pd.read_excel(JEAN, sheet_name='SAINT Analysis 2-3 repeats', usecols=['Bait', 'Prey', 'AvgSpec', 'SaintScore'])
jean_raw['Bait'] = jean_raw['Bait'].str.replace('R', 'RAB') # Change name to match other datasets
jean = jean_raw[jean_raw['SaintScore'] > saint_threshold] # Filter out low confidence interactions
jean = jean.pivot_table(index='Prey', columns='Bait', values='AvgSpec')
jean.index = jean.index.str.upper()
jean.fillna(0, inplace=True)


In [ ]:
## Wilson et al. (2023)
saint_threshold = 0.95

wilson_raw = pd.read_csv(WILSON, sep='\t')
wilson = wilson_raw[wilson_raw['SaintScore'] > saint_threshold] # Filter out low confidence interactions
wilson = wilson.pivot_table(index='Prey', columns='Bait', values='AvgSpec')
wilson.index = wilson.index.str.upper()
wilson.columns = [
    rab_conversion[rab] if rab in rab_conversion else rab for rab in wilson.columns
]
wilson.fillna(0, inplace=True)

# Extract gene ids and gene names for gaf file to convert Wilson identification
columns = [
    'db', 'id', 'symbol', 'qualifier', 'go_id', 'db_reference', 'evidence_code',
    'with_from', 'aspect', 'db_object_name', 'db_object_synonym', 'db_object_type',
    'taxon', 'date', 'assigned_by' 'annotation_extension', 'gene_product_form_id'
]
gaf = pd.read_csv(GAF('human'), sep='\t', comment='!', index_col=False, names=columns)


name_mapper = dict(zip(gaf['id'], gaf['symbol']))

wilson.index = wilson.index.map(name_mapper).str.upper()
print(wilson.index.isna().sum(), "gene ids could not be mapped to gene names")
wilson.dropna(inplace=True)

### Interactors across studies

In [ ]:
# Get Rab which are common to all datasets
common = list(set.intersection(
    *map(set, [jean.columns, wilson.columns])
))

jean = jean[jean[common].sum(axis=1) != 0][common]
wilson = wilson[wilson[common].sum(axis=1) != 0][common]

common

In [ ]:
print(jean.index.shape[0], "different interactors were identified in Jean dataset")
print(wilson.index.shape[0], "different interactors were identified in Wilson dataset")

In [ ]:
# Create dict of datasets for simpler data handling
genes_datasets = {
    'This study': jean,
    'Wilson et al. (2023)': wilson
}

# Plot upset of common interactors
genes_data = {
    "This study": {
        "Rab4": jean.index[jean['RAB4'].astype(bool)].unique().values,
        "Rab11": jean.index[jean['RAB11'].astype(bool)].unique().values,
        "Rab25": jean.index[jean['RAB25'].astype(bool)].unique().values,
    },
    "Wilson et al." : {
        "Rab4": wilson.index[wilson['RAB4'].astype(bool)].unique().values,
        "Rab11": wilson.index[wilson['RAB11'].astype(bool)].unique().values,
        "Rab25": wilson.index[wilson['RAB25'].astype(bool)].unique().values
        
    },
}


In [ ]:
# All interactors Venn diagram
fig, ax = plt.subplots(figsize=(6, 6))
venn2(
      (
            set([interactor for interactors in genes_data['This study'].values() for interactor in interactors]),
            set([interactor for interactors in genes_data['Wilson et al.'].values() for interactor in interactors])
      ),
      set_labels=('This study', 'Wilson et al.'),
      set_colors=('#709B92', '#a84c4cff'),
      ax=ax,
)
fig.suptitle("All baits", y=0.9, fontsize=16)
# fig.savefig('figures/wilson_all_interactors.svg')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 6))

# Get the maximum number of interactors in a category for normalization
num_interactors = []
for bait in genes_data['This study']:
    num = set(genes_data['This study'][bait]).union(set(genes_data['Wilson et al.'][bait]))
    num_interactors.append(len(num))
max_num_interactors = max(num_interactors)

ax_pos = [(0, 0), (0, 1), (1, 0)]
for bait, pos in zip(genes_data['This study'], ax_pos):
    ax = axes[pos]

    # Get the normalization value to have the same scale on each venn diagram
    actual_num_interactors = len(set(genes_data['This study'][bait]).union(set(genes_data['Wilson et al.'][bait])))
    normalize_to = np.sqrt(max_num_interactors / actual_num_interactors)
    genes_data['This study'][bait] = [gene for gene in genes_data['This study'][bait] if gene != '']
    venn2(
        (
            set(genes_data['This study'][bait]),
            set(genes_data['Wilson et al.'][bait])
        ),
        set_labels=('This study', 'Wilson et al.'),
        set_colors=('#709B92', '#a84c4cff'),
        ax=ax,
        # layout_algorithm=LayoutAlgorithm(normalize_to=normalize_to),
    )
    ax.set_xlim(-normalize_to, normalize_to)
    ax.set_ylim(-normalize_to, normalize_to)
    ax.set_title(bait)

axes[1, 1].axis('off')  # Hide the last subplot

# fig.savefig('figures/wilson_interactors_by_bait.svg', bbox_inches='tight')

### GO analysis

In [ ]:
# Get all interactors to extract GO terms from annotations
interactors = {
    'human': jean.index.tolist() + wilson.index.tolist(),
}

In [ ]:
# Dict to store Go id for each interactors for each GO term category
goa = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

# Get GO annotations for the identified interactors
for species, interacts in interactors.items():
    with gzip.open(GAF(species), 'rt') as f:
        for annotation in gafiterator.gafiterator(f):
            if annotation['DB_Object_Symbol'] not in interacts:
                continue
            
            aspect = annotation['Aspect']
            symbol = annotation['DB_Object_Symbol']
            if annotation['GO_ID'] not in goa[species][aspect][symbol]:
                goa[species][aspect][symbol].append(annotation['GO_ID'])
                
goa = defaultdict_to_regular(goa)

In [ ]:
# Extract GO terms for each dataset and each GO terms category and make an upset plot
go_categories = {
        'F': 'molecular_function',
        'C': 'cellular_component',
        'P': 'biological_process'
        }

# Get GO terms for each dataset and each GO terms category
go_data = {}
for category, category_name in go_categories.items():
    go_data[category_name] = {
            "This study": {
                "Rab4": get_go_terms(
                    goa['human'][category], jean.index[jean['RAB4'].astype(bool)].unique().values
                    ),
                "Rab11": get_go_terms(
                    goa['human'][category], jean.index[jean['RAB11'].astype(bool)].unique().values
                ),
                "Rab25": get_go_terms(
                    goa['human'][category], jean.index[jean['RAB25'].astype(bool)].unique().values
                ),
            },
            "Wilson et al.": {
                "Rab4": get_go_terms(
                    goa['human'][category], wilson.index[wilson['RAB4'].astype(bool)].unique().values
                ),
                "Rab11": get_go_terms(
                    goa['human'][category], wilson.index[wilson['RAB11'].astype(bool)].unique().values
                ),
                "Rab25": get_go_terms(
                    goa['human'][category], wilson.index[wilson['RAB25'].astype(bool)].unique().values
                )
            }
            
        }

In [ ]:
# Compare GO terms between studies (independantly of the bait)
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
ax_pos = [(0, 0), (0, 1), (1, 0)]

# Get maximum number of GO terms in a category for normalization
num_go_terms = []
for category in go_data:
    num = 0
    for bait in go_data[category]['This study']:
        num += len(set(go_data[category]['This study'][bait]).union(set(go_data[category]['Wilson et al.'][bait])))
    num_go_terms.append(num)
max_num_go_terms = max(num_go_terms)

for category, pos in zip(go_data.keys(), ax_pos):
    data = go_data[category]
    ax = axes[pos]

    jean_go = []
    wilson_go = []
    for bait in data["This study"]:
        jean_go.extend(data['This study'][bait])
        wilson_go.extend(data['Wilson et al.'][bait])
        
    # Get the normalization value to have the same scale on each venn diagram
    actual_num_go = len(set(jean_go).union(set(wilson_go)))
    normalize_to = np.sqrt(max_num_go_terms / actual_num_go) * 0.75 # Adjusted for better visualization
    venn2(
        (
            set(jean_go),
            set(wilson_go)
        ),
        set_labels=('This study', 'Wilson et al.'),
        set_colors=('#709B92', '#a84c4cff'),
        ax=ax,
        # layout_algorithm=LayoutAlgorithm(normalize_to=normalize_to),
    )
    ax.set_xlim(-normalize_to, normalize_to)
    ax.set_ylim(-normalize_to, normalize_to)
    ax.set_title(category)

axes[1, 1].axis('off')  # Hide the last subplot
# fig.savefig('figures/wilson_go_terms.svg', bbox_inches='tight')